## **Chroma**


### **Vector Store란?**
RAG( Retrieval‑Augmented Generation ) 파이프라인에서 **문서 청크→벡터 인덱싱→유사도 검색** 단계에 쓰임

**생성된 임베딩 벡터들을 효율적으로 저장하고 관리함**

| 이름 | 오픈소스/상용 | 배포 형태 | 언어 지원 | 특징 | 추천 사용 사례 |
|-------|--------------|-----------|-----------|-------|----------------|
| **FAISS** | OSS (Meta) | 라이브러리 (Python) | Python | 매우 빠른 로컬 검색, GPU 지원 | 단일 머신·빠른 PoC |
| **Chroma** | OSS | Embeddable (Python) | Python | 간단한 API, SQLite 백엔드 | 로컬 개발·실험 |
| **Qdrant** | OSS | Standalone 서버 | Python, Go, REST | 실시간 CRUD, 필터링, 하이퍼파라미터 튜닝 | 프로덕션 소규모 |
| **Milvus** | OSS | Standalone 클러스터 | Python, Go, REST | 대규모 분산, 고성능, GPU 가속 | 대용량·엔터프라이즈 |
| **Weaviate** | OSS | Standalone 클러스터 | Python, GraphQL, REST | 내장 NLP 모듈(객체 검색), 스키마 지원 | 메타데이터 검색 중심 |
| **Pinecone** | SaaS | Fully‑managed | REST, Python | 서버리스, 자동 스케일링, 모니터링 | 빠른 배포, 최소 운영 |
| **Zilliz Cloud** | SaaS | Managed Milvus | REST, Python | Milvus 기반, 클라우드 관리형 | Milvus 성능 + 관리 편의성 |
| **RedisVector** | OSS/Enterprise | Redis 모듈 | Redis CLI, Python | 단순 벡터 검색, Redis 에코시스템 | 기존 Redis 사용자 |
| **Elasticsearch** | OSS/Enterprise | Standalone | REST | 텍스트+벡터 하이브리드 검색 | 로그 + 벡터 동시 관리 |

---

### 🔍 선택 가이드

| 조건 | 추천 솔루션 |
|--------|------------|
| **빠른 로컬 PoC** | FAISS, Chroma |
| **경량 프로덕션** | Qdrant, RedisVector |
| **대규모 분산** | Milvus, Weaviate |
| **운영 부담 최소화** | Pinecone, Zilliz Cloud |
| **메타데이터 복합 쿼리** | Weaviate, Elasticsearch |

---

### **참고문서**

[LangChain 지원](https://python.langchain.com/v0.2/docs/integrations/vectorstores/)

In [1]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings

# Text splitter 정의
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

# Document loader 정의
loader1 = TextLoader("data/finance-keywords.txt")
loader2 = TextLoader("data/nlp-keywords.txt")

# 문서 분할 
split_doc1 = loader1.load_and_split(text_splitter)
split_doc2 = loader2.load_and_split(text_splitter)

# 문서 개수 확인
len(split_doc1), len(split_doc2)

(4, 7)

## **`from_documents` (Chroma) — 벡터 저장소 생성**

### **설명**  
`from_documents` 메서드는 Document 리스트로부터 Chroma 벡터 저장소를 생성함.

### **주요 파라미터**

|이름|타입|설명|기본값|
|---|---|---|---|
|documents|List[Document]|추가할 문서 리스트|—|
|embedding|Embeddings \| None|임베딩 함수|None|
|ids|List[str] \| None|문서별 고유 ID|None|
|collection_name|str|컬렉션 이름|—|
|persist_directory|str \| None|저장 디렉토리 (지정 시 디스크에 저장, 미지정 시 메모리)|None|
|client_settings|chromadb.config.Settings \| None|Chroma 클라이언트 설정|None|
|client|chromadb.Client \| None|기존 Chroma 클라이언트 인스턴스|None|
|collection_metadata|Dict \| None|컬렉션 메타데이터|None|

### **동작**

1. 내부적으로 `from_texts` 호출  
2. Document.page_content → 텍스트, metadata → 메타데이터로 사용  
3. `persist_directory` 지정 시 디스크에 영구 저장  

### **반환값**

- **Chroma** 인스턴스 (생성된 벡터 저장소)

- 생성된 Chroma 벡터 저장소 인스턴스 생성시 documents 매개변수로 Document 리스트를 전달
- embedding 에 활용할 임베딩 모델을 지정
- namespace 의 역할을 하는 collection_name 을 지정


In [2]:
# 임베딩 모델
embeddings = OpenAIEmbeddings(model = 'text-embedding-3-small')
 
# 저장경로
db_path = './db/'

# DB 생성
db = Chroma.from_documents(
    
    documents = split_doc1,
    embedding = embeddings,
    persist_directory = db_path,   # 지정시 파일로 저장
    collection_name = 'my_db'
    
) 

In [3]:
# 문서 로드 
persist_db = Chroma(
    persist_directory = db_path,
    embedding_function = embeddings,
    collection_name = 'my_db'
)

# 저장된 데이터 확인
persist_db.get()['documents'][0][:300]

'S&P 500\n\n정의: S&P 500은 미국 주식 시장에 상장된 500개의 대형 기업의 주가를 종합한 지수입니다. 이는 미국 경제와 주식 시장의 전반적인 상황을 나타내는 주요 지표로 사용됩니다.\n예시: 애플, 마이크로소프트, 아마존과 같은 대형 기술 기업들이 S&P 500에 포함되어 있습니다.\n연관키워드: 주식 시장, 지수, 대형주\n\nMarket Capitalization\n\n정의: 시가총액은 회사의 발행 주식 수와 현재 주가를 곱한 값으로, 회사의 전체 가치를 나타냅니다.\n예시: 애플의 시가총액이 2조 달러를 넘어서면서 S&P 50'

## **`from_texts` (Chroma) — 벡터 저장소 생성**

### **설명**  
`from_texts` 메서드는 문자열 리스트로부터 Chroma 벡터 저장소를 생성함.

### **주요 파라미터**

|이름|타입|설명|기본값|
|---|---|---|---|
|texts|List[str]|추가할 텍스트 리스트|—|
|embedding|Embeddings \| None|임베딩 함수|None|
|metadatas|List[dict] \| None|텍스트별 메타데이터|None|
|ids|List[str] \| None|문서 ID (미제공 시 UUID 자동 생성)|None|
|collection_name|str|컬렉션 이름|`_LANGCHAIN_DEFAULT_COLLECTION_NAME`|
|persist_directory|str \| None|저장 디렉토리 (지정 시 디스크, 미지정 시 메모리)|None|
|client_settings|chromadb.config.Settings \| None|Chroma 클라이언트 설정|None|
|client|chromadb.Client \| None|기존 클라이언트 인스턴스|None|
|collection_metadata|Dict \| None|컬렉션 메타정보|None|

### **동작**

- `persist_directory` 지정 시 디스크에 영구 저장  
- `ids` 미제공 시 UUID 자동 생성  

In [9]:
# 텍스트 문자열을 DB에 저장 
db2 = Chroma.from_texts(
    
    ['안녕하세요', '집에 가고싶어요'],
    embedding = embeddings,
    persist_directory = db_path,   # 지정시 파일로 저장
    collection_name = 'my_db2'
    
)

# 문서 로드 
persist_db2 = Chroma(
    persist_directory = db_path,
    embedding_function = embeddings,
    collection_name = 'my_db2'
)


# 저장된 데이터 확인
persist_db2.get()

{'ids': ['c507a198-75d3-476f-8d79-9426124ee5c0',
  '64ecac3d-17ce-459c-b62c-fd943b4b97b7',
  'b707cdb2-3805-4cd2-8f9e-753661abe270',
  '78cc0a6b-ef37-4496-83d4-333d0a85b697',
  '46c037ac-a8b9-49cf-9edf-d76e56d5e621',
  '88c9f320-4b5a-40a7-a626-ffaafaed3b53',
  '1d620c5d-76ea-49ee-bf89-eef0888d0593',
  'ed429c9b-1878-48e5-9ad9-c112bb5db1b3',
  '11a6207b-69c5-4103-b6c0-a4a3ad8aaaad',
  'b5eca084-3961-4e30-a1fa-87d38d0b8f6d',
  'fe53ce35-23ee-4876-8fc4-34a611cd4eff',
  '4a9a8d03-7e93-4cfa-8902-14279f38d89a'],
 'embeddings': None,
 'documents': ['안녕하세요',
  '집에 가고싶어요',
  '안녕하세요',
  '집에 가고싶어요',
  '안녕하세요',
  '집에 가고싶어요',
  '안녕하세요',
  '집에 가고싶어요',
  '안녕하세요',
  '집에 가고싶어요',
  '안녕하세요',
  '집에 가고싶어요'],
 'uris': None,
 'data': None,
 'metadatas': [None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

## **Chroma — 유사도 검색**

### **설명**  
주어진 쿼리와 가장 유사한 문서를 반환함.

### **주요 파라미터**

|이름|타입|설명|기본값|
|---|---|---|---|
|query|str|검색할 쿼리 텍스트|—|
|k|int|반환할 결과 개수|4|
|filter|Dict[str,str] \| None|메타데이터 필터링|None|

### **특징**

- `k` 조정으로 반환 개수 변경 가능  
- `filter` 로 특정 메타데이터 조건 적용  
- **점수 없이 문서만 반환**  
  - 점수 정보 필요 시 → `similarity_search_with_score`

In [12]:
# k=1 : 가장 유사한 문서 1개 반환
db.similarity_search('경기순환주에 대해 알려줘', k=1)  

[Document(id='0a7c4fa4-3c7d-42f8-83ab-f95dd1313d02', metadata={'source': 'data/finance-keywords.txt'}, page_content='정의: 경기순환주는 경제 상황에 따라 실적이 크게 변동하는 기업의 주식을 말합니다.\n예시: 포드, 제너럴 모터스와 같은 자동차 기업들은 S&P 500에 포함된 대표적인 경기순환주입니다.\n연관키워드: 경제 사이클, 섹터 분석, 투자 타이밍\n\nDefensive Stocks\n\n정의: 방어주는 경기 변동에 상관없이 안정적인 실적을 보이는 기업의 주식을 의미합니다.\n예시: 프록터앤갬블, 존슨앤존슨과 같은 생활필수품 기업들은 S&P 500 내 대표적인 방어주로 꼽힙니다.\n연관키워드: 안정적 수익, 저변동성, 리스크 관리')]

In [15]:
# filter 에 metadata 정보를 활용하여 검색 결과를 필터링
db.similarity_search(
    "경기순환주 알려줘", filter={"source": "data/finance-keywords.txt"}, k=2
)

[Document(id='0a7c4fa4-3c7d-42f8-83ab-f95dd1313d02', metadata={'source': 'data/finance-keywords.txt'}, page_content='정의: 경기순환주는 경제 상황에 따라 실적이 크게 변동하는 기업의 주식을 말합니다.\n예시: 포드, 제너럴 모터스와 같은 자동차 기업들은 S&P 500에 포함된 대표적인 경기순환주입니다.\n연관키워드: 경제 사이클, 섹터 분석, 투자 타이밍\n\nDefensive Stocks\n\n정의: 방어주는 경기 변동에 상관없이 안정적인 실적을 보이는 기업의 주식을 의미합니다.\n예시: 프록터앤갬블, 존슨앤존슨과 같은 생활필수품 기업들은 S&P 500 내 대표적인 방어주로 꼽힙니다.\n연관키워드: 안정적 수익, 저변동성, 리스크 관리'),
 Document(id='3d437797-c278-4e8e-8377-bb9ce6f34d1c', metadata={'source': 'data/finance-keywords.txt'}, page_content='정의: 시장 변동성은 주식 시장의 가격 변동 폭을 나타내는 지표입니다.\n예시: VIX 지수(변동성 지수)가 상승하면 S&P 500 지수의 변동성이 높아질 것으로 예상됩니다.\n연관키워드: 리스크 관리, 투자 심리, 헤지 전략\n\nEquity Research\n\n정의: 주식 리서치는 기업의 재무 상태, 사업 모델, 경쟁력 등을 분석하여 투자 의사 결정을 돕는 활동입니다.\n예시: 골드만삭스의 애널리스트들이 S&P 500 기업들에 대한 분기별 실적 전망을 발표했습니다.\n연관키워드: 투자 분석, 기업 가치평가, 시장 전망\n\nCorporate Governance\n\n정의: 기업 지배구조는 기업의 경영과 통제에 관한 시스템과 프로세스를 의미합니다.\n예시: S&P 500 기업들 중 이사회의 다양성을 높이는 기업들이 증가하고 있습니다.\n연관키워드: 주주 권리, ESG, 기업 윤리\n\nMergers and Acquisit

In [16]:
# filter 사용
db.similarity_search(
    "TF IDF 에 대하여 알려줘", filter={"source": "data/nlp-keywords.txt"}, k=2
)

# db에 nlp-keywords.txt를 추가하지 않아 빈 리스트 반환

[]

## **`add_documents` (Chroma) — 벡터 저장소에 문서 추가/업데이트**

### **설명**  
기존 컬렉션에 Document 객체 리스트를 추가하거나 업데이트함.

### **주요 파라미터**

|이름|타입|설명|
|---|---|---|
|documents|List[Document]|추가할 Document 리스트|
|ids|List[str] (선택)|문서별 ID (제공 시 Document.id 우선)|

### **동작 규칙**

- Document.page_content → 텍스트, metadata → 메타데이터로 사용  
- `ids` 제공 시 해당 ID 사용  
- Document에 `id`가 있고 `ids` 미제공 시 Document.id 사용  
- `ids` 길이 ≠ documents 길이 → `ValueError`

In [20]:
from langchain_core.documents import Document

db.add_documents(
    [
        Document(
            page_content = '새로운 문서 추가',
            metadata = {'source': 'data/my_text.txt'},
            id = "1"
            
        )
    ]
)

# id로 문서 조회
db.get('1')

{'ids': ['1'],
 'embeddings': None,
 'documents': ['새로운 문서 추가'],
 'uris': None,
 'data': None,
 'metadatas': [{'source': 'data/my_text.txt'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

## **`add_texts` (Chroma) — 텍스트 임베딩 후 추가**

### **설명**  
문자열 리스트를 임베딩해 Chroma 컬렉션에 추가함.

### **주요 파라미터**

|이름|타입|설명|기본값|
|---|---|---|---|
|texts|Iterable[str]|추가할 텍스트 리스트|—|
|metadatas|List[dict] \| None|텍스트별 메타데이터|None|
|ids|List[str] \| None|문서 ID (미제공 시 UUID 자동 생성)|None|

### **동작**

1. `ids` 미제공 → UUID 자동 생성  
2. 임베딩 함수로 텍스트 → 벡터  
3. 메타데이터 없는 항목 → `{}`로 채움  
4. 컬렉션에 **upsert** (텍스트, 벡터, 메타데이터) 수행

In [27]:
# 신규 데이터를 추가합니다. 이때 기존의 id=1 의 데이터는 덮어씀
db.add_texts(
    ["이전에 추가한 Document 를 덮어쓰겠습니다.", "덮어쓴 결과가 어떤가요?"],
    metadatas=[{"source": "mydata.txt"}, {"source": "mydata.txt"}],
    ids=["1", "2"],
)

db.get('1')

{'ids': ['1'],
 'embeddings': None,
 'documents': ['이전에 추가한 Document 를 덮어쓰겠습니다.'],
 'uris': None,
 'data': None,
 'metadatas': [{'source': 'mydata.txt'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

## ****`delete` (Chroma) — 문서 삭제****  

### 설명  
컬렉션에서 지정한 ID의 문서를 제거함.  

### 주요 파라미터  

|이름|타입|설명|기본값|
|---|---|---|---|
|ids|List[str] \| None|삭제할 문서 ID 리스트|None|

### 동작  
- `ids` 제공 시 해당 ID의 문서를 컬렉션에서 삭제  
- `ids`가 `None`이면 아무 작업도 수행하지 않음  

In [28]:
# id 1 삭제
db.delete(ids=["1"])

# 문서 조회
db.get(["1", "2"])

{'ids': ['2'],
 'embeddings': None,
 'documents': ['덮어쓴 결과가 어떤가요?'],
 'uris': None,
 'data': None,
 'metadatas': [{'source': 'mydata.txt'}],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

## ****`reset_collection` (Chroma) — 컬렉션 초기화****  

### 설명  
벡터 저장소의 컬렉션을 초기화함.  

### 주요 파라미터  

|이름|타입|설명|기본값|
|---|---|---|---|
|collection_name|str|초기화할 컬렉션 이름|—|

### 동작  
- 지정한 `collection_name`의 모든 데이터를 삭제하고 초기 상태로 복원  
- 기존 컬렉션이 존재하지 않으면 새로 생성  

In [29]:
# 컬렉션 초기화
db.reset_collection()

# 초기화 후 문서 조회
db.get()

{'ids': [],
 'embeddings': None,
 'documents': [],
 'uris': None,
 'data': None,
 'metadatas': [],
 'included': [<IncludeEnum.documents: 'documents'>,
  <IncludeEnum.metadatas: 'metadatas'>]}

## ****`as_retriever` (Chroma) — 벡터 저장소를 검색기로 변환****  

### 설명  
벡터 저장소를 기반으로 `VectorStoreRetriever` 생성함.  

### 주요 파라미터  

|이름|타입|설명|기본값|
|---|---|---|---|
|search_type|str \| None|검색 유형 ("similarity", "mmr", "similarity_score_threshold")|None|
|search_kwargs|Dict \| None|검색 함수에 전달할 추가 인자|None|
|k|int|반환할 문서 수|4|
|score_threshold|float \| None|최소 유사도 임계값|None|
|fetch_k|int|MMR 알고리즘에 전달할 문서 수|20|
|lambda_mult|float|MMR 결과의 다양성 조절 (0~1)|0.5|
|filter|Dict \| None|문서 메타데이터 필터링|None|

### 동작  
- 벡터 저장소를 `VectorStoreRetriever` 로 변환  
- `search_type` 에 따라 검색 방식 결정  
- `filter` 사용 시 특정 메타데이터 조건을 만족하는 문서만 검색  
- `MMR` 방식 선택 시 `fetch_k`, `lambda_mult` 를 적용하여 다양성 조절  

In [30]:
# DB 생성
db = Chroma.from_documents(
    documents=split_doc1 + split_doc2,
    embedding=embeddings,
    collection_name="nlp",
)

# 기본값 4개
retriever = db.as_retriever()
retriever.invoke("Word2Vec 에 대하여 알려줘")

[Document(id='87f24b02-143a-43b1-8530-4a2dc1dad4b0', metadata={'source': 'data/nlp-keywords.txt'}, page_content='정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하는 라이브러리입니다. 이는 연구자와 개발자들이 쉽게 NLP 작업을 수행할 수 있도록 돕습니다.\n예시: HuggingFace의 Transformers 라이브러리를 사용하여 감정 분석, 텍스트 생성 등의 작업을 수행할 수 있습니다.\n연관키워드: 자연어 처리, 딥러닝, 라이브러리\n\nDigital Transformation\n\n정의: 디지털 변환은 기술을 활용하여 기업의 서비스, 문화, 운영을 혁신하는 과정입니다. 이는 비즈니스 모델을 개선하고 디지털 기술을 통해 경쟁력을 높이는 데 중점을 둡니다.\n예시: 기업이 클라우드 컴퓨팅을 도입하여 데이터 저장과 처리를 혁신하는 것은 디지털 변환의 예입니다.\n연관키워드: 혁신, 기술, 비즈니스 모델\n\nCrawling\n\n정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.\n예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.\n연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진\n\nWord2Vec\n\n정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large Language Model)\n\n정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을 의미합니다. 이러한 모델은 다양한 자연어 이

In [32]:
# 파라미터 설정

retriever = db.as_retriever(
    search_type = 'mmr',
    search_kwargs = {'K' : 6,
                     'lambda_mult' : 0.25,
                    'fetch_k' : 10}
)

retriever.invoke("Word2Vec 에 대하여 알려줘")

[Document(id='87f24b02-143a-43b1-8530-4a2dc1dad4b0', metadata={'source': 'data/nlp-keywords.txt'}, page_content='정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하는 라이브러리입니다. 이는 연구자와 개발자들이 쉽게 NLP 작업을 수행할 수 있도록 돕습니다.\n예시: HuggingFace의 Transformers 라이브러리를 사용하여 감정 분석, 텍스트 생성 등의 작업을 수행할 수 있습니다.\n연관키워드: 자연어 처리, 딥러닝, 라이브러리\n\nDigital Transformation\n\n정의: 디지털 변환은 기술을 활용하여 기업의 서비스, 문화, 운영을 혁신하는 과정입니다. 이는 비즈니스 모델을 개선하고 디지털 기술을 통해 경쟁력을 높이는 데 중점을 둡니다.\n예시: 기업이 클라우드 컴퓨팅을 도입하여 데이터 저장과 처리를 혁신하는 것은 디지털 변환의 예입니다.\n연관키워드: 혁신, 기술, 비즈니스 모델\n\nCrawling\n\n정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.\n예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.\n연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진\n\nWord2Vec\n\n정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large Language Model)\n\n정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을 의미합니다. 이러한 모델은 다양한 자연어 이

In [33]:
# MMR 알고리즘을 위해 더 많은 문서를 가져오되 상위 2개만 반환
retriever = db.as_retriever(search_type="mmr", search_kwargs={"k": 2, "fetch_k": 10})
retriever.invoke("Word2Vec 에 대하여 알려줘")

[Document(id='87f24b02-143a-43b1-8530-4a2dc1dad4b0', metadata={'source': 'data/nlp-keywords.txt'}, page_content='정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하는 라이브러리입니다. 이는 연구자와 개발자들이 쉽게 NLP 작업을 수행할 수 있도록 돕습니다.\n예시: HuggingFace의 Transformers 라이브러리를 사용하여 감정 분석, 텍스트 생성 등의 작업을 수행할 수 있습니다.\n연관키워드: 자연어 처리, 딥러닝, 라이브러리\n\nDigital Transformation\n\n정의: 디지털 변환은 기술을 활용하여 기업의 서비스, 문화, 운영을 혁신하는 과정입니다. 이는 비즈니스 모델을 개선하고 디지털 기술을 통해 경쟁력을 높이는 데 중점을 둡니다.\n예시: 기업이 클라우드 컴퓨팅을 도입하여 데이터 저장과 처리를 혁신하는 것은 디지털 변환의 예입니다.\n연관키워드: 혁신, 기술, 비즈니스 모델\n\nCrawling\n\n정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.\n예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.\n연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진\n\nWord2Vec\n\n정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large Language Model)\n\n정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을 의미합니다. 이러한 모델은 다양한 자연어 이

In [34]:
# 특정 임계값 이상의 유사도를 가진 문서만 검색
retriever = db.as_retriever(
    search_type="similarity_score_threshold", search_kwargs={"score_threshold": 0.8}
)

retriever.invoke("Word2Vec 에 대하여 알려줘")

No relevant docs were retrieved using the relevance score threshold 0.8


[]

In [35]:
# 가장 유사한 단일 문서만 검색
retriever = db.as_retriever(search_kwargs={"k": 1})

retriever.invoke("Word2Vec 에 대하여 알려줘")

[Document(id='87f24b02-143a-43b1-8530-4a2dc1dad4b0', metadata={'source': 'data/nlp-keywords.txt'}, page_content='정의: HuggingFace는 자연어 처리를 위한 다양한 사전 훈련된 모델과 도구를 제공하는 라이브러리입니다. 이는 연구자와 개발자들이 쉽게 NLP 작업을 수행할 수 있도록 돕습니다.\n예시: HuggingFace의 Transformers 라이브러리를 사용하여 감정 분석, 텍스트 생성 등의 작업을 수행할 수 있습니다.\n연관키워드: 자연어 처리, 딥러닝, 라이브러리\n\nDigital Transformation\n\n정의: 디지털 변환은 기술을 활용하여 기업의 서비스, 문화, 운영을 혁신하는 과정입니다. 이는 비즈니스 모델을 개선하고 디지털 기술을 통해 경쟁력을 높이는 데 중점을 둡니다.\n예시: 기업이 클라우드 컴퓨팅을 도입하여 데이터 저장과 처리를 혁신하는 것은 디지털 변환의 예입니다.\n연관키워드: 혁신, 기술, 비즈니스 모델\n\nCrawling\n\n정의: 크롤링은 자동화된 방식으로 웹 페이지를 방문하여 데이터를 수집하는 과정입니다. 이는 검색 엔진 최적화나 데이터 분석에 자주 사용됩니다.\n예시: 구글 검색 엔진이 인터넷 상의 웹사이트를 방문하여 콘텐츠를 수집하고 인덱싱하는 것이 크롤링입니다.\n연관키워드: 데이터 수집, 웹 스크래핑, 검색 엔진\n\nWord2Vec\n\n정의: Word2Vec은 단어를 벡터 공간에 매핑하여 단어 간의 의미적 관계를 나타내는 자연어 처리 기술입니다. 이는 단어의 문맥적 유사성을 기반으로 벡터를 생성합니다.\n예시: Word2Vec 모델에서 "왕"과 "여왕"은 서로 가까운 위치에 벡터로 표현됩니다.\n연관키워드: 자연어 처리, 임베딩, 의미론적 유사성\nLLM (Large Language Model)\n\n정의: LLM은 대규모의 텍스트 데이터로 훈련된 큰 규모의 언어 모델을 의미합니다. 이러한 모델은 다양한 자연어 이

In [36]:
# 특정 메타데이터 필터 적용
retriever = db.as_retriever(
    search_kwargs={"filter": {"source": "data/finance-keywords.txt"}, "k": 2}
)
retriever.invoke("ESG 에 대하여 알려줘")

[Document(id='f6b786cb-d86e-4784-affa-24edb52709ec', metadata={'source': 'data/finance-keywords.txt'}, page_content='Price-to-Earnings Ratio (P/E Ratio)\n\n정의: 주가수익비율(P/E)은 주가를 주당순이익으로 나눈 값으로, 기업의 가치를 평가하는 데 사용되는 지표입니다.\n예시: 아마존의 P/E 비율이 높은 것은 투자자들이 회사의 미래 성장 가능성을 높게 평가하고 있다는 것을 의미합니다.\n연관키워드: 주식 가치평가, 투자 분석, 성장주\n\nQuarterly Earnings Report\n\n정의: 분기별 실적 보고서는 기업이 3개월마다 발표하는 재무 성과와 사업 현황에 대한 보고서입니다.\n예시: 애플의 분기별 실적 발표는 전체 기술 섹터와 S&P 500 지수에 큰 영향을 미칩니다.\n연관키워드: 기업 실적, 투자자 관계, 재무 분석\n\nIndex Fund\n\n정의: 인덱스 펀드는 S&P 500과 같은 특정 지수의 구성과 성과를 그대로 추종하는 투자 상품입니다.\n예시: 바운가드 S&P 500 ETF는 S&P 500 지수를 추종하는 대표적인 인덱스 펀드입니다.\n연관키워드: 패시브 투자, ETF, 포트폴리오 관리\n\nMarket Weight\n\n정의: 시장 가중치는 특정 기업이나 섹터가 전체 지수에서 차지하는 비중을 나타냅니다.\n예시: 기술 섹터는 S&P 500 지수에서 가장 큰 시장 가중치를 차지하고 있습니다.\n연관키워드: 포트폴리오 구성, 섹터 분석, 자산 배분\n\nGrowth Stock\n\n정의: 성장주는 평균 이상의 높은 성장률을 보이는 기업의 주식을 의미합니다.\n예시: 페이스북(메타)과 같은 기술 기업들은 S&P 500에 포함된 대표적인 성장주로 꼽힙니다.\n연관키워드: 고성장 기업, 기술주, 투자 전략\n\nValue Stock\n\n정의: 가치주는 현재 시장 가치가 내재 가치보다 낮다고 평가되는 기업의 주식을